<a href="https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這支 colab notebook 是為了選取剪輯 [news-digest](https://news-digest.vercel.app/) 網站當天的新聞而寫的。只根據新聞分類(catetories)剪輯。

執行步驟如下：
1. 按下 Install 下那個 code cell 前面的 ► 按鈕來安裝用到的模組
2. 根據需要，填表執行後面的 Widgets UI 或 Forms

需要更細的檢索查詢，有如下兩個選擇
- 直接在 [news-digest 網站](https://news-digest.vercel.app/)上搜尋或點選 hash-tags
- 或者點擊 [news_query.ipynb](https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_query.ipynb) 執行多條件檢索


### Install 

In [ ]:
import os

fns = ['clip.py', 'line.py', 'op.py']

for fn in fns:
    if os.path.exists(fn):
        os.remove(fn)
for fn in fns:
    url = f'https://raw.githubusercontent.com/YorkJong/news-digest/main/api/{fn}'
    !wget $url

import clip
import line

#-------------------------------------------------------------------------------
import ipywidgets as widgets

def create_menu(content):
    options = clip.get_categories(content)
    categories = widgets.SelectMultiple(
        options=options,
        #value=[options[0]],
        rows=len(options),
        description='Categories',
        disabled=False,
    )

    with_hashtags = widgets.Checkbox(
        value=True,
        description='with hashtags',
        disabled=False,
    )

    with_headers = widgets.Checkbox(
        value=True,
        description='with headers',
        disabled=False,
    )

    flags = widgets.VBox([with_hashtags, with_headers])
    output = widgets.Output()
    display(widgets.HBox([categories, flags]), output)

    def on_value_change(change):
        with output:
            #print(change['new'])
            output.clear_output()
            lines = clip.get_lines_of_categories(categories.value, content, with_hashtags.value, with_headers.value)
            print('\n'.join(lines))

    categories.observe(on_value_change, names='value')
    with_hashtags.observe(on_value_change, names='value')
    with_headers.observe(on_value_change, names='value')

### Select categories with Jupyter Widgets
- 按下面那個 code cell 前面的 ► 按鈕，以列出完整的分類(Catetories)選單
- macOS 下操作可按Cmd後用滑鼠複選（Windows 系統則按下Ctrl鍵後以滑鼠複選）
  - 全選可按 Cmd+A (macOS) 或者 Ctrl+A (Windows)

In [ ]:
create_menu(clip.get_latest_journal())

### Select Categories with the Colab Forms

In [ ]:
#@title Single Selection { run: "auto", display-mode: "form" }
category = "AI" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = False #@param {type:"boolean"}

content = clip.get_latest_journal()
lines = clip.get_lines_of_categories([category], content, with_hashtags, with_headers)
text = '\n'.join(lines)
print(text)

In [ ]:
#@title Multiple Selections { run: "auto", display-mode: "form" }
first_category = "Tesla & SpaceX; Vehicle" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
last_category = "Finance" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = True #@param {type:"boolean"}

content = clip.get_latest_journal()
categories = clip.get_categories(content)
categories = clip.get_sublist(categories, first_category, last_category)
lines = clip.get_lines_of_categories(categories, content, with_hashtags, with_headers)
text = '\n'.join(lines)
print(text)


### See Also
- [news_query.ipynb (from GitHub)](https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_query.ipynb)